In [ ]:
import requests
import pandas as pd
import time
import numpy as np

In [ ]:
df = pd.read_csv(f'us_station_id.csv')
coutry = 'usa'
df_weather = {}
base_url = "https://www.ncei.noaa.gov/access/services/data/v1"

In [ ]:
for index, row in df.iterrows():
    if row['lat'] in df_weather:
        print(f'The data of {row['lat']} has already been obtained.')
    else:
        params = {
            "dataset": "daily-summaries",
            "stations": f"{row['STATION']}", 
            "startDate": "1900-01-01",
            "endDate": "2025-02-27",
            "format": "json"
        }
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            print(f"Obtaining weather...")
            data = response.json()
            if not data:
                print(f"There's no weather data.")
            else:
                print('Succeeded!')
                df_weather[f'{row['lat']}'] = data
                time.sleep(1)
        else:
            print("Error:", response.status_code, response.text)

In [ ]:
dfs = [] 
for location, json_data in df_weather.items():
    df = pd.DataFrame(json_data)  
    dfs.append(df) 

result_df = pd.concat(dfs, ignore_index=True)
result_df.drop_duplicates(inplace=True)

out_df = df.merge(result_df, on='STATION', how='left')
out_df = out_df[['DATE', 'TMAX', 'TMIN', 'PRCP', 'lat', 'long']]
output_df = out_df[~((out_df.TMAX.isna()) & (out_df.TMAX.isna()))]
output_df.to_csv(f'{coutry}_weather.csv', index=False)

In [ ]:
for 